In [1]:
import pandas as pd
import os
import sys
import pickle
import datetime
import time
import re
from sqlalchemy import create_engine
from sqlalchemy import text
import sqlalchemy
import joblib
from collections import Counter
import numpy as np

In [2]:
TOKENIZERS_PARALLELISM=False
TF_ENABLE_ONEDNN_OPTS=0

In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

2026-02-19 18:30:23.160541: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-19 18:30:23.193875: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-19 18:30:23.979783: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
STOP_WORDS = { "the",
    "and", "or", "the", "a", "an", "to", "of", "in", "on", "for", "with",
    "by", "at", "from", "as", "is", "are", "this", "that", 'tv', 'collection',
    'episodes' ,'episode', 'и', 'ну', 'не', 'ссср','shorts', 'y', 'kidsmania',
    'un', 'el', 'de', 'be'
}

In [5]:
def tokenize_clean(text):
    tokens = re.findall(r"\b[^\W_]+\b", text.lower(), flags=re.UNICODE)
    return [ t for t in tokens if not t.isdigit() and t not in STOP_WORDS ]

In [6]:
df = pd.read_excel('markuped.xlsx')
ids = df.yt_reel_id.to_list()
ids_str = "'" + "','".join(ids) + "'"

In [7]:
conn_string = os.environ['PG_DB2']
db = create_engine(conn_string.replace('multdb', 'youtube'))
conn = db.connect()
req1 = '''select iyr.yt_reel_id, reel_name, yt_ch_name, cartoon, brand_id, product_id,language from intl_yt_reels iyr 
join intl_yt_channels2reels iyrcr 
on iyr.yt_reel_id = iyrcr.yt_reel_id 
join intl_yt_channels iyc 
on iyrcr.yt_ch_id = iyc.yt_ch_id 
where iyr.yt_reel_id in ({}) '''.format(ids_str)
df_sql = pd.read_sql(text(req1), conn)
conn.close()

In [8]:
df_sql['name'] = df_sql.yt_ch_name + ' '+df_sql.reel_name
df_sql["tokens"] = df_sql["name"].apply(tokenize_clean)
texts2 = df_sql['tokens'].str.join(' ').to_list()
labels2 = df_sql['brand_id'].to_list()

In [9]:
pipeline = joblib.load("pipeline.pkl")
y_pred = pipeline.predict(texts2)

In [10]:
print(classification_report(labels2, y_pred))
print(confusion_matrix(labels2, y_pred))

              precision    recall  f1-score   support

                   0.00      0.00      0.00      38.0
      BR0071       0.00      0.00      0.00       0.0
      BR0175       0.00      0.00      0.00       0.0
      BR0176       0.00      0.00      0.00       0.0
      BR0186       0.00      0.00      0.00       0.0
      BR0192       0.00      0.00      0.00       0.0
      BR0251       0.00      0.00      0.00       0.0

    accuracy                           0.00      38.0
   macro avg       0.00      0.00      0.00      38.0
weighted avg       0.00      0.00      0.00      38.0

[[ 0 13  2 13  1  7  2]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]]


/home/a.gankov/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/a.gankov/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/a.gankov/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/a.gankov/.local/lib/python3.9/site-p

In [11]:
final = pd.concat([df_sql,pd.DataFrame(y_pred)], axis =1 )

In [12]:
final.to_excel('markuped+multiclass_new.xlsx', index = False)